# V7 Quantum Training - Colab via VS Code

**SIRALAMA: Hucreleri 1'den 8'e kadar sirayla calistir. Atlama!**

## Hucre 1: GPU Kontrolu

In [ ]:
!nvidia-smi
import torch
print(f"CUDA: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

## Hucre 2: Google Drive Mount

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Hucre 3: Veriyi Lokal Diske Kopyala

Drive uzerinden okuma cok yavas. Once veriyi Colab'in hizli diskine kopyaliyoruz.

In [ ]:
import shutil, os

LOCAL_TRAIN = '/content/local_data/train'
LOCAL_TEST = '/content/local_data/test'

if not os.path.exists(LOCAL_TRAIN):
    print("Train verisi kopyalaniyor...")
    shutil.copytree('/content/drive/MyDrive/set/train', LOCAL_TRAIN)
    print(f"Train: {len(os.listdir(LOCAL_TRAIN))} dosya")
else:
    print(f"Train zaten mevcut: {len(os.listdir(LOCAL_TRAIN))} dosya")

if not os.path.exists(LOCAL_TEST):
    print("Test verisi kopyalaniyor...")
    shutil.copytree('/content/drive/MyDrive/set/test', LOCAL_TEST)
    print(f"Test: {len(os.listdir(LOCAL_TEST))} dosya")
else:
    print(f"Test zaten mevcut: {len(os.listdir(LOCAL_TEST))} dosya")

print("Kopyalama tamam!")

## Hucre 4: Repo Clone + Dependencies

In [ ]:
import os

REPO_DIR = '/content/Quanvolutional-Neural-Network'

if os.path.exists(REPO_DIR):
    !cd {REPO_DIR} && git pull
else:
    !git clone https://github.com/necatiincekara/Quanvolutional-Neural-Network.git {REPO_DIR}

os.chdir(REPO_DIR)
print(f"Dizin: {os.getcwd()}")

!pip install pennylane pennylane-lightning-gpu "numpy>=2.0" tqdm matplotlib -q
print("Dependencies OK")

## Hucre 5: Path'leri Lokal Diske Yonlendir + Ortam Dogrula

In [ ]:
from src import config

# Drive yerine lokal diski kullan (10x daha hizli)
config.TRAIN_PATH = '/content/local_data/train'
config.TEST_PATH = '/content/local_data/test'

print(f"Platform:  Colab={config.IS_COLAB}")
print(f"Compute:   {config.DEVICE}")
print(f"Quantum:   {config.QUANTUM_DEVICE}")
print(f"Train:     {config.TRAIN_PATH} ({len(os.listdir(config.TRAIN_PATH))} dosya)")
print(f"Test:      {config.TEST_PATH} ({len(os.listdir(config.TEST_PATH))} dosya)")

assert config.DEVICE == 'cuda', "GPU yok!"
print("\nHer sey hazir!")

## Hucre 6: Model Test (30 saniye)

In [ ]:
from src.trainable_quantum_model import create_enhanced_model

model = create_enhanced_model(circuit_type='data_reuploading', num_classes=44)
total = sum(p.numel() for p in model.parameters())
q = sum(p.numel() for n, p in model.named_parameters() if 'quanv' in n)
print(f"V7 Model: {total} params (quantum: {q}, classical: {total-q})")

x = torch.randn(2, 1, 32, 32).to(config.DEVICE)
model = model.to(config.DEVICE)
with torch.no_grad():
    out = model(x)
print(f"Forward pass: {x.shape} -> {out.shape}")
print("Model GPU'da calisiyor!")
del model, x, out  # Bellegi temizle

## Hucre 7: TRAINING BASLAT (~5-6 saat)

**Ilk 5 dakika**: Quantum kernel compile (yavas, normal)

**Sonra izle**:
- `quantum grad mean` > 1e-4 ise → saglikli, devam
- `quantum grad mean` = 0 ise → **DURDUR**
- 20 batch sonra loss dusmuyorsa → **DURDUR**

In [ ]:
from src.enhanced_training import run_enhanced_training
import time

start = time.time()

best_acc, test_acc = run_enhanced_training(
    circuit_type='data_reuploading',
    num_epochs=3
)

elapsed = time.time() - start
hours = int(elapsed // 3600)
minutes = int((elapsed % 3600) // 60)
print(f"\nToplam sure: {hours}s {minutes}dk")
print(f"En iyi val accuracy: {best_acc:.2f}%")
print(f"Test accuracy: {test_acc:.2f}%")

## Hucre 8: Sonuclari Drive'a Kaydet

In [ ]:
import shutil, glob

drive_backup = '/content/drive/MyDrive/quanv_results/v7/'
os.makedirs(drive_backup, exist_ok=True)

if os.path.exists('models/best_v7_model.pth'):
    shutil.copy('models/best_v7_model.pth', drive_backup)
    print(f"Model: {drive_backup}best_v7_model.pth")

exp_dirs = glob.glob('experiments/v7_*')
if exp_dirs:
    latest = max(exp_dirs, key=os.path.getmtime)
    dest = os.path.join(drive_backup, os.path.basename(latest))
    if os.path.exists(dest):
        shutil.rmtree(dest)
    shutil.copytree(latest, dest)
    print(f"Experiment: {dest}")

print(f"Sonuclar kaydedildi: {drive_backup}")